In [1]:
import os
import zipfile
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever
# print(PyPDFLoader)
# print(RecursiveCharacterTextSplitter)
# print(FAISS)
# print(HuggingFaceEmbeddings)


#Extract zip files from data folder
DATA_PATH='Data/'

def extract_zip_files(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.zip'):
            zip_path=os.path.join(folder_path, filename)
            with zipfile.ZipFile(zip_path, 'r')as zip_ref:
                zip_ref.extractall(folder_path)
                print(f'Extracted: {filename}')



extract_zip_files(DATA_PATH)




#Step-1: load all pdfs (even inside subfolders)
def load_pdf_files(data_path):
    all_documents=[]
    for root, dirs, files in os.walk(data_path):
        for file in files:
            if file.endswith('.pdf'):
                pdf_path=os.path.join(root, file)
                loader=PyPDFLoader(pdf_path)
                documents = loader.load()
                all_documents.extend(documents)
    print(f"📄 Loaded {len(all_documents)} PDF pages.")
    return all_documents

documents=load_pdf_files(DATA_PATH)

# def load_pdf_files(data):
#     loader=DirectoryLoader(data,
#                            glob='*.pdf',
#                            loader_cls=PyPDFLoader)
#     documents=loader.load()
#     return documents

# documents=load_pdf_files(data=DATA_PATH)
# print('Length of PDF Pages', len(documents))

#Step: 2
def create_chunks(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks=create_chunks(extracted_data=documents)
print('Length of Text Chunks', len(text_chunks))

Extracted: Banking_Data.zip
Extracted: Regulated_Products.zip
📄 Loaded 2521 PDF pages.
Length of Text Chunks 16007


In [3]:
def get_embedding_model():
    embedding_model=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embedding_model

embedding_model=get_embedding_model()

In [4]:
from langchain.vectorstores import FAISS
vectorstore=FAISS.from_documents(text_chunks, embedding_model)

In [5]:
vectorstore_retriever=vectorstore.as_retriever(search_kwargs={"k": 3})
vectorstore_retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022079B70B00>, search_kwargs={'k': 3})

In [6]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [7]:
keyword_retriever=BM25Retriever.from_documents(text_chunks)

In [8]:
keyword_retriever.k=3

In [ ]:
ensemble_retriever=EnsembleRetriever(retrievers=[vectorstore_retriever, keyword_retriever], weights=[0.5, 0.5] )

In [12]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq  # NEW import
from dotenv import load_dotenv
load_dotenv() 

# Step 1: Setup LLM (Mistral-7B via Groq)
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

def load_llm():
    llm = ChatGroq(
        api_key=GROQ_API_KEY,
        model_name="llama3-70b-8192",  # You can also try "llama3-70b-8192" or "gemma-7b-it"
        temperature=0.5,
        max_tokens=512
    )
    return llm

# Step 2: Create Custom Prompt
CUSTOM_PROMPT_TEMPLATE = """
You are a helpful, professional, and knowledgeable banking assistant specializing in the UAE banking system. Your role is to provide clear, accurate, and up-to-date information about:

- UAE Central Bank regulations
- Islamic banking and conventional banking in UAE
- Accounts (current, savings, salary)
- Credit cards, loans, and financing options
- KYC (Know Your Customer) requirements
- Bank transfers (local, international, WPS)
- Digital banking and mobile apps
- Fees, charges, and interest rates
- Banking for residents vs. non-residents
- Bank working hours and public holidays
- Account opening for individuals and businesses

Always tailor your responses using the following guidelines:

1. **Context-Aware:** Respond based on the banking laws, financial institutions, and practices specific to the United Arab Emirates (UAE).
2. **Concise Yet Informative:** Give to-the-point responses but provide links to official resources if more detail is needed.
3. **Professional Tone:** Maintain a respectful, formal tone with a customer-centric approach.
4. **Do Not Provide Legal or Investment Advice.**
5. **If Unsure:** If the user's question requires legal interpretation or up-to-date verification (like current interest rates), suggest checking with the relevant UAE bank or Central Bank website.

Context:
{context}

Question:
{question}

Start the answer directly. No small talk please.
"""

def set_custom_prompt(custom_prompt_template):
    prompt = PromptTemplate(
        template=custom_prompt_template,
        input_variables=["context", "question"]
    )
    return prompt


In [13]:
#Create  Normal RetrievalQA Chain
normal_qa_chain = RetrievalQA.from_chain_type(
    llm=load_llm(),
    chain_type="stuff",
    retriever=vectorstore_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": set_custom_prompt(CUSTOM_PROMPT_TEMPLATE)}
)

In [17]:
#Create  Hybrid RetrievalQA Chain
hybrid_qa_chain = RetrievalQA.from_chain_type(
    llm=load_llm(),
    chain_type="stuff",
    retriever=ensemble_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": set_custom_prompt(CUSTOM_PROMPT_TEMPLATE)}
)

In [18]:
response1=normal_qa_chain.invoke("tell me how can i get home loans?")

In [19]:
print(response1.get("result"))

To obtain a home loan in the UAE, you can follow these steps:

1. **Check your eligibility**: Ensure you meet the lender's eligibility criteria, such as age, income, and credit score requirements.
2. **Choose a lender**: Select a mortgage loan provider, such as a bank or financial institution, that offers home loans in the UAE. Compare their interest rates, fees, and terms to find the best option for you.
3. **Gather required documents**: Typically, you'll need to provide documents such as:
	* Valid UAE residence visa
	* Emirates ID
	* Salary certificate or proof of income
	* Bank statements
	* Property details (if purchasing a new property)
4. **Apply for the loan**: Submit your application to the chosen lender, either online, through a branch, or via a mortgage consultant.
5. **Loan processing and approval**: The lender will review your application, verify your documents, and assess your creditworthiness. They may also conduct a property valuation (if applicable).
6. **Loan offer and

In [20]:
response2=hybrid_qa_chain.invoke("tell me how can i get home loans?")

In [21]:
response2

{'query': 'tell me how can i get home loans?',
 'result': "To obtain a home loan in the UAE, you can follow these steps:\n\n1. **Check your eligibility**: Ensure you meet the lender's criteria, including age, income, employment, and credit score requirements.\n2. **Choose a lender**: Select a mortgage loan provider, such as a bank or financial institution, that offers home loans in the UAE. You can compare rates, terms, and conditions among different lenders.\n3. **Gather required documents**: Typically, you'll need to provide identification documents (e.g., passport, visa, Emirates ID), income proof (e.g., salary certificate, payslips), and property details (e.g., title deed, property valuation report).\n4. **Apply for pre-approval**: Submit your application to get pre-approved for a home loan, which will give you an estimate of how much you can borrow.\n5. **Select a property**: Find a property that meets your needs and budget, and ensure it's free from any encumbrances.\n6. **Finali

In [22]:
print(response2.get("result"))

To obtain a home loan in the UAE, you can follow these steps:

1. **Check your eligibility**: Ensure you meet the lender's criteria, including age, income, employment, and credit score requirements.
2. **Choose a lender**: Select a mortgage loan provider, such as a bank or financial institution, that offers home loans in the UAE. You can compare rates, terms, and conditions among different lenders.
3. **Gather required documents**: Typically, you'll need to provide identification documents (e.g., passport, visa, Emirates ID), income proof (e.g., salary certificate, payslips), and property details (e.g., title deed, property valuation report).
4. **Apply for pre-approval**: Submit your application to get pre-approved for a home loan, which will give you an estimate of how much you can borrow.
5. **Select a property**: Find a property that meets your needs and budget, and ensure it's free from any encumbrances.
6. **Finalize the loan application**: Once you've selected a property, comple